# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [7]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [8]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [10]:
# 切分訓練資料及測試資料
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.03705,20.0,3.33,0,0.4429,6.968,37.2,5.2447,5,216
2,0.02543,55.0,3.78,0,0.484,6.696,56.4,5.7321,5,370
3,0.1,34.0,6.09,0,0.433,6.982,17.7,5.4917,7,329
4,0.05083,0.0,5.19,0,0.515,6.316,38.1,6.4584,5,224
5,2.924,0.0,19.58,0,0.605,6.101,93.0,2.2834,5,403
6,8.64476,0.0,18.1,0,0.693,6.193,92.6,1.7912,24,666
7,0.62356,0.0,6.2,1,0.507,6.879,77.7,3.2721,8,307
8,0.20608,22.0,5.86,0,0.431,5.593,76.5,7.9549,7,330
9,0.19539,0.0,10.81,0,0.413,6.245,6.2,5.2873,4,305


In [12]:
model = GLM.lm(@formula(Tax ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Tax ~ 1 + Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad

Coefficients:
─────────────────────────────────────────────────────────────────────────────────
               Estimate  Std. Error    t value  Pr(>|t|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────────
(Intercept)  207.128      46.721      4.4333      <1e-4   115.275      298.981
Crim          -0.136769    0.401007  -0.341063    0.7332   -0.925144     0.651606
Zn             0.862077    0.171379   5.03025     <1e-6     0.525148     1.199
Indus          7.58278     0.737489  10.2819      <1e-21    6.13289      9.03268
Chas         -27.8187     12.9974    -2.14033     0.0329  -53.3714      -2.26601
NOx           28.9862     48.8302     0.593611    0.5531  -67.0135     124.986
Rm           -10.259    

In [14]:
predict(model, test)

101-element Array{Union{Missing, Float64},1}:
 279.2204865030187
 322.4465983418051
 335.47810872171567
 286.91534367389033
 408.2589744720681
 666.1529021442059
 308.5318202799943
 356.3525620225795
 303.17423394046693
 331.472250923178
 354.6154373661571
 407.3850311659534
 665.0573702185167
   ⋮
 333.85841106818367
 340.9414596234055
 324.0972673890625
 247.6102418589149
 405.7688143170254
 427.55548765485935
 310.9816954015244
 248.9620170394742
 666.2463855961685
 355.7399246802795
 672.240259086072
 277.2145303256274

In [16]:
# 模型評估
GLM.r²(model)

0.8862096587762619

In [17]:
GLM.adjr²(model)

0.88361696745724